In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from collections import deque
import schedule
import time
import boto3
import os
import warnings
warnings.filterwarnings("ignore")

In [ ]:
os.environ['AWS_ACCESS_KEY_ID'] = 'your-access-key'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'your-secret-key'
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

In [ ]:
np.random.seed(42)
times = pd.date_range(start='2023-01-01', periods=200000, freq='T')
seasonal = 100 + 20 * np.sin(2 * np.pi * np.arange(len(times)) / 1440)
trend = 0.001 * np.arange(len(times))
noise = np.random.normal(0, 5, size=len(times))
spikes = (np.random.rand(len(times)) < 0.001) * np.random.randint(50, 200, size=len(times))
requests = seasonal + trend + noise + spikes

df = pd.DataFrame({
    'timestamp': times,
    'requests': requests
})
df.set_index('timestamp', inplace=True)
df = df.sort_index()

In [ ]:
scaler = MinMaxScaler()
df['req_scaled'] = scaler.fit_transform(df[['requests']])

SEQ_LEN = 60
def create_sequences(data, seq_len=60):
    X, y = [], []
    for i in range(len(data) - seq_len):
        X.append(data[i:i+seq_len])
        y.append(data[i+seq_len])
    return np.array(X), np.array(y)

X, y = create_sequences(df['req_scaled'].values, SEQ_LEN)
split = int(0.8 * len(X))
X_train, X_val = X[:split], X[split:]
y_train, y_val = y[:split], y[split:]
X_train = X_train[..., np.newaxis]
X_val = X_val[..., np.newaxis]

In [ ]:
model = Sequential([
    LSTM(64, input_shape=(SEQ_LEN,1), return_sequences=True),
    LSTM(32),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, batch_size=512)

In [ ]:
def forecast_and_detect(latest_seq, threshold=0.3):
    pred_scaled = model.predict(np.expand_dims(latest_seq, axis=0))[0,0]
    pred = scaler.inverse_transform([[pred_scaled]])[0,0]
    actual = scaler.inverse_transform([[latest_seq[-1,0]]])[0,0]
    anomaly = abs(actual - pred) > threshold * actual
    return pred, anomaly

In [ ]:
class AutoScaler:
    def __init__(self, cooldown=120, min_nodes=1, max_nodes=10):
        self.current_nodes = min_nodes
        self.last_scale_ts = time.time()
        self.cooldown = cooldown
        self.max_nodes = max_nodes
        self.cap_per_node = 200

    def decide(self, predicted_req):
        now = time.time()
        if now - self.last_scale_ts < self.cooldown:
            return self.current_nodes
        needed = int(np.ceil(predicted_req / self.cap_per_node))
        needed = max(self.current_nodes, min(needed, self.max_nodes))
        if needed != self.current_nodes:
            self.scale_to(needed)
            self.last_scale_ts = now
        return self.current_nodes

    def scale_to(self, target_nodes):
        ec2 = boto3.resource('ec2', region_name='us-east-1')
        instances = list(ec2.instances.filter(
            Filters=[
                {'Name': 'tag:AutoScaleGroup', 'Values': ['SmartScaler']},
                {'Name': 'instance-state-name', 'Values': ['running', 'stopped']}
            ]))
        running = [i for i in instances if i.state['Name'] == 'running']
        stopped = [i for i in instances if i.state['Name'] == 'stopped']
        diff = target_nodes - len(running)
        print(f"[Scaler] Target: {target_nodes}, Running: {len(running)}, Stopped: {len(stopped)}")
        if diff > 0:
            for inst in stopped[:diff]:
                inst.start()
        elif diff < 0:
            for inst in running[diff:]:
                inst.stop()
        self.current_nodes = target_nodes

In [ ]:
latest_seq = deque(df['req_scaled'].values[-SEQ_LEN:], maxlen=SEQ_LEN)
auto_scaler = AutoScaler()

def get_latest_request():
    value = df['req_scaled'].sample(1).values[0]
    timestamp = pd.Timestamp.now()
    return value, timestamp

def real_time_tick():
    new_val, ts = get_latest_request()
    latest_seq.append(new_val)
    seq = np.array(latest_seq)[..., np.newaxis]
    if len(seq) == SEQ_LEN:
        pred, anomaly = forecast_and_detect(seq)
        pred_actual = scaler.inverse_transform([[pred]])[0,0]
        current_nodes = auto_scaler.decide(pred_actual)
        print(f"[{ts}] Predicted Req: {pred_actual:.1f}, Anomaly: {anomaly}, Nodes: {current_nodes}")

schedule.every(1).minutes.do(real_time_tick)

print("[INFO] Starting real-time auto-scaling loop (press Ctrl+C to stop)")
while True:
    schedule.run_pending()
    time.sleep(1)